In [10]:
import os
import sys
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.tools import DuckDuckGoSearchResults
from langchain_community.vectorstores import FAISS


In [13]:
df = pd.read_csv('/home/aragy/Huma.ai-assessment/Dataset/cleaned_dataset.csv')

In [30]:
from langchain.embeddings import FastEmbedEmbeddings


class FastEmbedEmbeddingsSingleton:
    _instance = None

    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super(FastEmbedEmbeddingsSingleton, cls).__new__(cls)
            cls._instance._init_once(*args, **kwargs)
        return cls._instance

    def _init_once(self, *args, **kwargs):

        self.embeddings = FastEmbedEmbeddings()

In [33]:
fastembed = FastEmbedEmbeddings()

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 64726.91it/s]


In [27]:
def store_chunks(chunks: list[str]):
    embeddings = fastembed
    db = FAISS.from_texts(chunks, embeddings)
    db.save_local('/home/aragy/Huma.ai-assessment/FAISSDB')

In [28]:
def search_similar_chunks(query: str, k: int = 5):
    embeddings = fastembed
    db = FAISS.load_local('/home/aragy/Huma.ai-assessment/FAISSDB', embeddings,allow_dangerous_deserialization=True)
    results = db.similarity_search(query, k)
    return [result.page_content for result in results]

In [14]:
df.head()

,Unnamed: 0,country,requester_type,product,indication,question,channel,date_time_open,date_time_closed,answer_solution,response_time_hours,question_token_count,answer_token_count
0,0,UK,HCP,Keytruda,NSCLC,What are the common side effects of Keytruda?,email,2023-06-27 17:00:00,2023-07-21 18:00:00,"Common side effects include fatigue, nausea, a...",577.0,9,12
1,1,US,Researcher,Keytruda,NSCLC,Can Keytruda cause immune-related adverse effe...,email,2023-06-08 08:00:00,2023-07-06 12:30:00,"Yes, Keytruda can cause immune-related adverse...",676.5,7,17
2,2,UK,HCP,Keytruda,NSCLC,Is Keytruda safe for pregnant women?,call,2023-03-06 04:00:00,2023-04-19 15:00:00,Keytruda is not recommended for use during pre...,1067.0,7,16
3,3,UK,Researcher,Keytruda,NSCLC,What should patients report immediately while ...,email,2023-02-05 10:30:00,2023-03-05 06:00:00,Patients should report any new or worsening sy...,667.5,10,21
4,4,US,Researcher,Keytruda,NSCLC,Are there any known interactions between Keytr...,call,2023-03-14 09:30:00,2023-04-16 18:30:00,"Yes, Keytruda can interact with steroids and c...",801.0,11,18


In [23]:
documents = [row['question']+' '+row['answer_solution'] for _,row in df.iterrows()]

In [32]:
store_chunks(documents)

AttributeError: 'NoneType' object has no attribute 'embed'

In [7]:
embeddings = FastEmbedEmbeddingsSingleton().embeddings
db = FAISS.load_local('./FAISSDB', embeddings,allow_dangerous_deserialization=True)

Number of embeddings in the index: 49
